# Explore and Clean Data 2

In [1]:
import math as mt

import numpy as np

import pandas as pd

import pandas_profiling

import re

import mpu.io

# Enables interactive figures
# import mpld3

%matplotlib notebook
import matplotlib.pyplot as plt

#mpld3.enable_notebook()
plt.rcParams['figure.figsize'] = [9.5, 6]

## Import previously saved data

Data was saved in a pickled format. Let's reimport it.

The dataframe "df5": 
- consists of all the Features (=columns, =variables) that are not constant and that are not all NA
- it has 347 columns (27 were droped)
- the datetime and datetimetz variables were read with the correct dtype

The dictionary "constants":
- consists of all columns that are actually constant

The columns 'qc_entnahme_datum', 'quiver_url' and 'quiver_encoded_query' where droped since they only contained missing values ('NaN')

In [2]:
df5 = pd.read_pickle('./Data/process_data_df5.pkl')
constants = mpu.io.read('./Data/process_constants.json')

## Divide and conquer
Let's check the different dtypes: 

In [3]:
print(*df5.dtypes.unique(),sep = '\n')

datetime64[ns]
float64
object
bool
int64
datetime64[ns, UTC]


I believe we can trust that the 'datetime' dtypes were correctely recogized.

Issues:
- [ ] Some 'float' variables might actually be 'integers', 'booleans' or 'categorical'
- [ ] Some 'object' variables might actually also be better interpreted as 'categorical'
- [x] We should check what the 'integer' variables are.
- [ ] Check whether all '%' values are between 0 and 100%.

Minor issues:
- [ ] Variable names (column names) should be consistent (either all in German or all in English?) 
    - [x] '_%' instead of '_in_percentage', 
    - [x] '_date' instead of '_datum'
    - [x] 'bad-wechsel' and 'badwechsel'
- [ ] Be careful with typos e.g 'windung' instead of 'winding'

## fix column names

In [176]:
# Get a list with all variable (column) names
oldNames = list(df5.columns)

# Let's change all 'datum's to English or to German?
dictRename = {
    'date':'datum',
    'badwechsel':'bad-wechsel',
    'in_percentage':'%',
    'windung':'winding'
#    'beschichtete_rollenlange':'coated_roll_length'
}


# Initialize variable newNames
newNames = oldNames

for old, new in dictRename.items():
    newNames = list(
        map(
            lambda name: re.sub(old,new,name),
            newNames
            )
        )


df5 = df5.rename(columns=dict(zip(oldNames,newNames)))

variables = list(df5.columns)        

In [177]:
# Check the new column names:
# [s for s in newNames if 'winding' in s]
df5.columns

Index(['pa_datum', 'pa_ps_beschichtete_rollenlange_m',
       'pa_beschichtete_rollenlange_m', 'pa_bahn-geschwin-digkeit_m/min',
       'pa_bad-wechsel_m-pda', 'pa_bad-wechsel_hw1', 'pa_bad-wechsel_hw2',
       'pa_bad-wechsel_chlor', 'pa_bad-wechsel_hw3',
       'pa_raum_temperatur_start_c',
       ...
       'chlor_con_median', 'f_danfugt_bs_median', 'f_danfugt_as_median',
       'f_codfely_median', 'temp_median', 'ps_f_auftragswerk_median',
       'ps_dicke_median', 'reaction_start', 'reaction_end', 'derived_datum'],
      dtype='object', length=347)

### constants
'constants' provide important information from reference values:

In [178]:
constants

{'component': ['MEB'],
 'p_netting': [0],
 'pa_amin-trockner_temperatur_cofely_ref_c': [20.0],
 'pa_ce-capro_lactam_ref_%': [0.0],
 'pa_ctmc_richtwert_ref_%': [0.12],
 'pa_raum_temperatur_start_ref_high_c': [27.0],
 'pa_raum_temperatur_start_ref_low_c': [18.0],
 'pa_staub-sauger_ref_low_0_aus_>0_an': [0.0],
 'pa_temperatur_chlorbad_start_ref_c': [20.0],
 'pa_temperatur_m-pda-bad_ref_high': [22.0],
 'pa_temperatur_m-pda-bad_ref_low': [17.0],
 'ps_auftragsbank_temperatur_start_ref_c': [19.0],
 'ps_auftragswerk_feuchtigkeit_ref_high_%': [65.0],
 'ps_auftragswerk_feuchtigkeit_ref_low_%': [45.0],
 'ps_bad_temperatur_ref_high_c': [23.0],
 'ps_bad_temperatur_ref_low_c': [17.0],
 'ps_c_losung_ref_wt_%': [31.0],
 'ps_dicke_as_ref_high_micro_m': [150.0],
 'ps_dicke_as_ref_low_micro_m': [140.0],
 'ps_gap_ref_high_micro_m': [280.0],
 'ps_gap_ref_low_micro_m': [250.0],
 'ps_raum-feuchtigkeit_start_ref_high_%': [100.0],
 'ps_raum-feuchtigkeit_start_ref_low_%': [45.0],
 'ps_raum_temperatur_start_ref_

## integer

In [179]:
df5.select_dtypes('integer').head()

,pa_rollen_seit_letztem_bad-wechsel_mpda,pa_rollen_seit_letztem_bad-wechsel_chlor,qc_faktorkonzentration,p_product_size
0,2,2,1,8
1,2,2,1,8
2,2,2,1,8
3,2,2,1,8
4,2,2,1,8


In [180]:
df5.select_dtypes('integer').nunique()

pa_rollen_seit_letztem_bad-wechsel_mpda     7
pa_rollen_seit_letztem_bad-wechsel_chlor    7
qc_faktorkonzentration                      2
p_product_size                              3
dtype: int64

### boolean

In [181]:
df5.select_dtypes(bool).head()

,pa_bad-wechsel_m-pda,pa_bad-wechsel_hw1,pa_bad-wechsel_hw2,pa_bad-wechsel_chlor,pa_bad-wechsel_hw3,pa_bad-wechsel_mpda_nach,pa_bad-wechsel_chlor_nach
0,False,False,True,False,False,False,False
1,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,True,False,False,False,False
4,False,False,True,False,False,False,False


In [182]:
df5.select_dtypes(bool).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118687 entries, 0 to 118686
Data columns (total 7 columns):
pa_bad-wechsel_m-pda         118687 non-null bool
pa_bad-wechsel_hw1           118687 non-null bool
pa_bad-wechsel_hw2           118687 non-null bool
pa_bad-wechsel_chlor         118687 non-null bool
pa_bad-wechsel_hw3           118687 non-null bool
pa_bad-wechsel_mpda_nach     118687 non-null bool
pa_bad-wechsel_chlor_nach    118687 non-null bool
dtypes: bool(7)
memory usage: 811.4 KB


Let's check all columns containing 'wechsel'.
Some columns have only '0's and '1's. We believe they are booleans!

In [183]:
# Get all columns containing the string 'wechsel'
wechselColumns = [s for s in variables if 'wechsel' in s]

# Check which of these columns contain only '0's ans '1's and change their dtype to boolean
for col in df5.loc[:,wechselColumns]:
    # print(pd.unique(df5[col]))
    # print(set(pd.unique(df5[col])) <= set([0,1]))
    if set(pd.unique(df5[col])) <= set([0,1]):
        df5[col] = df5[col].astype('bool')


In [184]:
df5.loc[:,wechselColumns].head()

,pa_bad-wechsel_m-pda,pa_bad-wechsel_hw1,pa_bad-wechsel_hw2,pa_bad-wechsel_chlor,pa_bad-wechsel_hw3,pa_bad-wechsel_mpda_nach,pa_bad-wechsel_chlor_nach,pa_rollen_seit_letztem_bad-wechsel_mpda,pa_rollen_seit_letztem_bad-wechsel_chlor
0,False,False,True,False,False,False,False,2,2
1,False,False,True,False,False,False,False,2,2
2,False,False,True,False,False,False,False,2,2
3,False,False,True,False,False,False,False,2,2
4,False,False,True,False,False,False,False,2,2


Let's do the same with the whole dataframe. Let's check whether we find variables whose only values are '0's and '1's:

In [186]:
# df5.select_dtypes(exclude=bool).head():

for col in df5.select_dtypes(exclude=bool):
    if set(pd.unique(df5[col])) <= set([0,1]):
        print(col)
    
# [col for col in variables if set(pd.unique(df5[col])) <= set([0,1]) ]

sc_l_ergebnis_p_nio
qc_faktorkonzentration
qc_nachkommadurchfluss


The variables 'sc_l_ergebnis_p_nio', 'qc_faktorkonzentration' and 'qc_nachkommadurchfluss' contain only '0's and '1's, but we cannot infer from their names wheter they should be treated as booleans of not. Therefore we'll leave them as they are for now.

### datetime and datetimetz

In [187]:
df5.select_dtypes(['datetime', 'datetimetz']).loc[50000:50006]

,pa_datum,ps_datum,ps_datum_coating,pa_datum_coating,qt_datum,pp_plan_actual_datum_coating,pp_plan_end_datum_winding,winding_begin_datum,winding_end_datum,qc_erfassungs_datum,...,qc_einlager_datum,sc_d_datum,sc_datum_generate,sc_l_datum_auto,sc_l_datum_hand,qc_datum_leak_test_values,qc_datum_product_properties,reaction_start,reaction_end,derived_datum
50000,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:10:47+00:00,...,2018-11-16 17:17:40+00:00,2018-11-15 12:10:47+00:00,2018-11-15 05:32:36+00:00,2018-11-15 05:32:36+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01
50001,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:18:49+00:00,...,2018-11-16 13:07:59+00:00,2018-11-15 12:18:49+00:00,2018-11-15 06:24:36+00:00,2018-11-15 06:24:36+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01
50002,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:18:45+00:00,...,2018-11-16 15:34:10+00:00,2018-11-15 12:18:45+00:00,2018-11-15 06:29:04+00:00,2018-11-15 06:29:04+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01
50003,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:25:10+00:00,...,2018-11-16 13:07:59+00:00,2018-11-15 12:25:10+00:00,2018-11-15 06:26:50+00:00,2018-11-15 06:26:50+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01
50004,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:25:06+00:00,...,2018-11-16 15:34:10+00:00,2018-11-15 12:25:06+00:00,2018-11-15 06:33:32+00:00,2018-11-15 06:33:32+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01
50005,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:30:15+00:00,...,2018-11-16 19:51:32+00:00,2018-11-15 12:30:15+00:00,2018-11-15 06:31:18+00:00,2018-11-15 06:31:18+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01
50006,2018-11-05,2018-11-03,2018-11-03,2018-11-04,2018-11-04,2018-11-04,2018-11-15,2018-11-14,2018-11-15,2018-11-15 12:30:09+00:00,...,2018-11-16 19:51:32+00:00,2018-11-15 12:30:09+00:00,2018-11-15 06:38:00+00:00,2018-11-15 06:38:00+00:00,NaT,2019-01-29 10:29:37+00:00,2013-01-08 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-05 00:00:00+00:00,2018-11-01


In [188]:
print(df5.select_dtypes(['datetime', 'datetimetz']).info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118687 entries, 0 to 118686
Data columns (total 21 columns):
pa_datum                        118687 non-null datetime64[ns]
ps_datum                        118687 non-null datetime64[ns]
ps_datum_coating                118661 non-null datetime64[ns]
pa_datum_coating                118687 non-null datetime64[ns]
qt_datum                        116303 non-null datetime64[ns]
pp_plan_actual_datum_coating    118687 non-null datetime64[ns]
pp_plan_end_datum_winding       118666 non-null datetime64[ns]
winding_begin_datum             117824 non-null datetime64[ns]
winding_end_datum               118621 non-null datetime64[ns]
qc_erfassungs_datum             118670 non-null datetime64[ns, UTC]
qc_verpackungs_datum            115035 non-null datetime64[ns, UTC]
qc_einlager_datum               114941 non-null datetime64[ns, UTC]
sc_d_datum                      118670 non-null datetime64[ns, UTC]
sc_datum_generate               90978 non-null dat

Issue:

- [ ] Variable 'sc_l_datum_hand' has only 76 non-null observations
    - Apparently it shows the dates and times when the side cut operation was done by hand?
    - If so, then we shouldn't discard this column, since it provides valuable quality related information.
    - [ ] We should check with the 'domain experts' what this column mean

In [191]:
pd.unique(df5['sc_l_datum_hand'])

<DatetimeArray>
[                      'NaT', '2018-10-19 15:02:29+00:00',
 '2019-02-05 14:46:03+00:00', '2019-03-08 12:39:21+00:00',
 '2018-07-24 14:33:17+00:00', '2018-07-24 15:13:26+00:00',
 '2018-11-09 09:31:59+00:00', '2019-03-28 17:22:23+00:00',
 '2019-03-28 17:25:03+00:00', '2019-03-28 17:30:03+00:00',
 '2019-03-28 17:33:23+00:00', '2018-12-09 00:54:11+00:00',
 '2019-03-27 13:45:41+00:00', '2019-06-23 13:56:36+00:00',
 '2019-06-26 02:19:04+00:00', '2018-10-25 22:59:34+00:00',
 '2018-10-26 01:02:28+00:00', '2018-10-26 01:17:34+00:00',
 '2018-10-26 15:02:04+00:00', '2018-10-18 14:05:51+00:00',
 '2018-10-18 13:50:56+00:00', '2018-10-18 15:29:09+00:00',
 '2018-10-18 17:09:54+00:00', '2018-10-28 06:11:06+00:00',
 '2018-10-28 06:13:42+00:00', '2018-10-28 06:23:02+00:00',
 '2018-10-28 06:26:09+00:00', '2018-10-28 06:45:13+00:00',
 '2018-10-28 08:14:05+00:00', '2018-10-28 17:28:54+00:00',
 '2018-10-28 19:19:41+00:00', '2019-02-01 07:15:37+00:00',
 '2019-02-01 08:58:36+00:00', '2019-08-2

Let's check the other columns with 'hand' and 'auto' in their names:

In [201]:
HandAutocols = [col for col in df5.columns if 'hand' in col or 'auto' in col]
print(HandAutocols)

noNas   = ~df5['sc_l_datum_hand'].isna()
onlyNas = ~noNas

df5.loc[:,HandAutocols].describe()

['pa_defects_hand_#', 'sc_l_leak_auto', 'sc_l_datum_auto', 'sc_l_leak_hand', 'sc_l_datum_hand']


,pa_defects_hand_#,sc_l_leak_auto,sc_l_leak_hand
count,118687.000000,118687.000000,118687.000000
mean,21.608693,0.073364,0.000838
std,339.492388,0.042482,0.186371
min,0.000000,-0.004397,0.000000
25%,0.000000,0.078259,0.000000
50%,0.000000,0.094966,0.000000
75%,0.000000,0.100242,0.000000
max,7125.000000,0.149484,55.000000


- Variable 'sc_l_leak_hand' is 0 when 'sc_l_datum_hand' is missing.
- Variable 'pa_defects_hand_#' appears to have a outlier (max = 7125)

In [9]:
df5.select_dtypes(['datetime', 'datetimetz']).describe()

,pa_datum,ps_datum,ps_date_coating,pa_date_coating,qt_datum,pp_plan_actual_date_coating,pp_plan_end_date_winding,windung_begin_date,winding_end_date,qc_erfassungs_datum,...,qc_einlager_datum,sc_d_datum,sc_datum_generate,sc_l_datum_auto,sc_l_datum_hand,qc_datum_leak_test_values,qc_datum_product_properties,reaction_start,reaction_end,derived_date
count,118687,118687,118661,118687,116303,118687,118666,117824,118621,118670,...,114941,118670,90978,89422,76,90978,118670,118687,118687,118670
unique,578,594,596,593,578,593,608,607,603,111951,...,6217,111951,46739,88769,71,22521,10,578,578,24
top,2018-01-28 00:00:00,2018-03-03 00:00:00,2019-01-28 00:00:00,2019-03-05 00:00:00,2018-09-06 00:00:00,2019-03-05 00:00:00,2019-02-05 00:00:00,2018-06-25 00:00:00,2018-03-12 00:00:00,2019-09-05 16:19:42+00:00,...,2018-09-21 08:28:36+00:00,2019-09-05 16:19:42+00:00,2019-06-10 02:22:50+00:00,2019-08-25 21:09:54+00:00,2019-09-24 19:08:53+00:00,2019-06-01 01:10:07+00:00,2013-01-08 00:00:00+00:00,2018-01-28 00:00:00+00:00,2018-01-28 00:00:00+00:00,2019-03-01 00:00:00
freq,900,834,964,984,1093,984,790,519,601,4,...,49,4,24,2,2,663,30701,900,900,7428
first,2018-01-04 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-03 00:00:00,2018-01-04 00:00:00,2018-01-03 00:00:00,2018-01-11 00:00:00,2017-05-31 00:00:00,2018-01-10 00:00:00,2018-01-09 23:14:40+00:00,...,2018-01-12 05:22:27+00:00,2018-01-09 23:14:40+00:00,2018-06-07 10:04:35+00:00,2018-06-07 12:17:35+00:00,2018-06-07 10:04:35+00:00,2018-10-15 15:21:03+00:00,2012-03-20 00:00:00+00:00,2018-01-04 00:00:00+00:00,2018-01-04 00:00:00+00:00,2018-01-01 00:00:00
last,2019-12-05 00:00:00,2019-11-16 00:00:00,2019-11-17 00:00:00,2019-12-05 00:00:00,2019-12-05 00:00:00,2019-12-05 00:00:00,2019-12-10 00:00:00,2019-12-06 00:00:00,2019-12-06 00:00:00,2019-12-06 10:47:57+00:00,...,2019-12-06 05:27:12+00:00,2019-12-06 10:47:57+00:00,2019-12-05 14:59:30+00:00,2019-12-06 09:07:23+00:00,2019-12-05 22:18:26+00:00,2019-12-06 09:05:01+00:00,2018-10-22 00:00:00+00:00,2019-12-05 00:00:00+00:00,2019-12-05 00:00:00+00:00,2019-12-01 00:00:00


### object

In [204]:
df5.select_dtypes(object).head()

,ps,nr,ps_lsg,pp_product_short_name,pp_plan_product,pp_actual_product_short_name,pp_actual_product,pp_actual_usage,winding_product_short_name,winding_product_type,winding_product_line,pa_tmc_gehalt_%,pa_ref,qc_serien_nummer,qc_barcode_leak_test_values,p_product,p_product_full_name,p_product_group,p_product_type,qc_pa_beschichtungsjahr
0,cb031d4b18ff,2891,230,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,9bec1f36ec0d,NaN,bc7e29194383,6f5dd5e75de0,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
1,cb031d4b18ff,2891,230,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,9bec1f36ec0d,NaN,bc7e29194383,e83198853aa3,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
2,cb031d4b18ff,2891,230,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,9bec1f36ec0d,NaN,bc7e29194383,0c6c47811c04,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
3,cb031d4b18ff,2891,230,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,9bec1f36ec0d,NaN,bc7e29194383,6b51542380df,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
4,cb031d4b18ff,2891,230,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,9bec1f36ec0d,NaN,bc7e29194383,58df9ba0a603,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W


In [205]:
df5.select_dtypes(object).tail()

,ps,nr,ps_lsg,pp_product_short_name,pp_plan_product,pp_actual_product_short_name,pp_actual_product,pp_actual_usage,winding_product_short_name,winding_product_type,winding_product_line,pa_tmc_gehalt_%,pa_ref,qc_serien_nummer,qc_barcode_leak_test_values,p_product,p_product_full_name,p_product_group,p_product_type,qc_pa_beschichtungsjahr
118682,3fa606fdd9e8,2892,231,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,6aa9aee40c62,NaN,0a0d4ada494a,55315e14346a,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
118683,3fa606fdd9e8,2892,231,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,6aa9aee40c62,NaN,0a0d4ada494a,5e840146da5b,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
118684,3fa606fdd9e8,2892,231,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,6aa9aee40c62,NaN,0a0d4ada494a,05ac5c0533e3,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
118685,3fa606fdd9e8,2892,231,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,6aa9aee40c62,NaN,0a0d4ada494a,b629f239aa69,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W
118686,3fa606fdd9e8,2892,231,6989995295da,1c39db15c26b,6989995295da,1c39db15c26b,use,6989995295da,6989995295da,6aa9aee40c62,NaN,0a0d4ada494a,44994043c050,NaN,1c39db15c26b,b3ccc5e5f9b5,4d6474389c69,6989995295da,W


In [12]:
df5.select_dtypes(object).loc[50000:50006]

,ps,nr,ps_lsg,pp_product_short_name,pp_plan_product,pp_actual_product_short_name,pp_actual_product,pp_actual_usage,winding_product_short_name,winding_product_type,winding_product_line,pa_tmc_gehalt_in_percentage,pa_ref,qc_serien_nummer,qc_barcode_leak_test_values,p_product,p_product_full_name,p_product_group,p_product_type,qc_pa_beschichtungsjahr
50000,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,79d813e4c6bf,2405711870910W0606WK31876,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W
50001,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,1ade0cad8b89,2405711870910W0606WK31877,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W
50002,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,cd97d5690539,2405711870910W0606WK31880,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W
50003,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,9f9ed3771287,2405711870910W0606WK31881,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W
50004,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,707deae1e2e0,2405711870910W0606WK31882,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W
50005,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,8f2bef211dd1,2405711870910W0606WK31883,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W
50006,aad34f59b337,3239,581,f8ba29e9058b,9b7853269545,6d2830b1e76d,9b7853269545,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,NaN,e0687f3265c3,482ad93b3873,2405711870910W0606WK31884,9b7853269545,73928f2577b5,4d6474389c69,6d2830b1e76d,W


Issues:
- [ ] 'nr', 'ps_lsg', 'pa_tmc_gehalt_in_percentage', 'qc_barcode_leak_test_values' were read as 'object', but apparently they're numerical.
    - [ ] 'nr' has empty strings of different lengths
    - [ ] 'ps_lsg' ValueError: Unable to parse string "061/062" at position 2399
    - [ ] 'pa_tmc_gehalt_%' ValueError: Unable to parse string "0,075/0,07" at position 46788
    - [ ] 'qc_barcode_leak_test_values' is actually 'object
- [ ] 'pp_actual_usage' appears to be a good categorical variable.
- [ ] The other 'object' variables that have less than 15 unique values could be also categorical. 
- [ ] 'qc_pa_beschichtungsjahr' has also some weird values: 'W', 'X' and NaN's

In [209]:
objs = df5.select_dtypes(object)
try: 
    pd.to_numeric(objs['nr'])
except:
    None

# pd.to_numeric(df5.select_dtypes)
# df5['qc_pa_beschichtungsjahr'].unique()


In [212]:
# Change 'nr' to numeric
df5.nr = pd.to_numeric(df5.nr,errors='coerse')

df5.select_dtypes(object)[df5.nr.isna()].head()

,ps,ps_lsg,pp_product_short_name,pp_plan_product,pp_actual_product_short_name,pp_actual_product,pp_actual_usage,winding_product_short_name,winding_product_type,winding_product_line,pa_tmc_gehalt_%,pa_ref,qc_serien_nummer,qc_barcode_leak_test_values,p_product,p_product_full_name,p_product_group,p_product_type,qc_pa_beschichtungsjahr
40337,6c7214167279,89,6989995295da,0ef124ce8628,6989995295da,0ef124ce8628,use,6989995295da,6989995295da,6aa9aee40c62,NaN,56c9d92c898e,87bcaec5bf6f,NaN,0ef124ce8628,2256f65c362b,4d6474389c69,6989995295da,W
40338,6c7214167279,89,6989995295da,0ef124ce8628,6989995295da,0ef124ce8628,use,6989995295da,6989995295da,6aa9aee40c62,NaN,56c9d92c898e,7023aca04de4,NaN,0ef124ce8628,2256f65c362b,4d6474389c69,6989995295da,W
40339,6c7214167279,89,6989995295da,0ef124ce8628,6989995295da,0ef124ce8628,use,6989995295da,6989995295da,6aa9aee40c62,NaN,56c9d92c898e,27f6d6102f93,NaN,0ef124ce8628,2256f65c362b,4d6474389c69,6989995295da,W
40340,6c7214167279,89,6989995295da,0ef124ce8628,6989995295da,0ef124ce8628,use,6989995295da,6989995295da,6aa9aee40c62,NaN,56c9d92c898e,8d8e67ff8873,NaN,0ef124ce8628,2256f65c362b,4d6474389c69,6989995295da,W
40341,6c7214167279,89,6989995295da,0ef124ce8628,6989995295da,0ef124ce8628,use,6989995295da,6989995295da,6aa9aee40c62,NaN,56c9d92c898e,35feb2a92b5a,NaN,0ef124ce8628,2256f65c362b,4d6474389c69,6989995295da,W


In [213]:
pd.to_numeric(objs['ps_lsg'])

ValueError: Unable to parse string "061/062" at position 2399

In [214]:
df5['ps_lsg'].unique()

array([230, 318, 342, 372, 530, 591, 617, 109, 157, 221, 254, 258, 39, 81,
       86, 149, 265, 316, 449, '449', '509', '574', '623', '043',
       '061/062', '147/148', '167', '273/274', '311', '500', '633', '063',
       '075', '084', '089', '207', '239', '240', '358/359', '75', '18',
       '477', '494', '499', '510', '582', '627', '50', '079', '135/136',
       '201', '217', '3', '148', '545', '037/36', '047', '062', '066',
       '115/116', '243', '318/320', '131', '168', '430', '598', '051',
       '149/150', '253', '271/272', '341', '347', '360', '355', '164',
       '537', '616', '628', '187', '255', '259', '273', '114', '190',
       '221', '416', '487', '556', '067', '094/095', '191', '220/221',
       '412', '340', '538', '634', '013/014', '088', '150', '188',
       '203/204', '206', '396/397', '32', '585', '625', '005', '133',
       '140', '244', '353', '392/393', '22', '82', '352', '597',
       '105/106', '142', '349/350', '203', '354', '523', '569', '022',
       '028'

'ps_lsg' seems to be useless. It has several strings of the form a/a+1 that cannot be parsed as numerical.

Possible solutions:
- [ ] drop column
- [ ] take the average of these values (a+0,5)

'pa_tmc_gehalt_%' has the same problem, but with only one value: '0,075/0,07'
    

In [16]:
df5['pa_tmc_gehalt_%'].unique()

array([nan, 0.06, 0.07, 0.075, 0.045, 0.08, 0.16, '0.07', '0.06', '0.16',
       '0,075/0,07', 0.05, 0.15], dtype=object)

In [17]:
pd.to_numeric(df5['pa_tmc_gehalt_%'])

ValueError: Unable to parse string "0,075/0,07" at position 46788

In [215]:
print(df5.select_dtypes(object).info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118687 entries, 0 to 118686
Data columns (total 19 columns):
ps                              118687 non-null object
ps_lsg                          118348 non-null object
pp_product_short_name           118687 non-null object
pp_plan_product                 118686 non-null object
pp_actual_product_short_name    118641 non-null object
pp_actual_product               118682 non-null object
pp_actual_usage                 118685 non-null object
winding_product_short_name      118687 non-null object
winding_product_type            118687 non-null object
winding_product_line            118687 non-null object
pa_tmc_gehalt_%                 43045 non-null object
pa_ref                          118687 non-null object
qc_serien_nummer                118670 non-null object
qc_barcode_leak_test_values     90978 non-null object
p_product                       118670 non-null object
p_product_full_name             118670 non-null object
p_product_g

In [216]:
df5.select_dtypes(object).describe()

,ps,ps_lsg,pp_product_short_name,pp_plan_product,pp_actual_product_short_name,pp_actual_product,pp_actual_usage,winding_product_short_name,winding_product_type,winding_product_line,pa_tmc_gehalt_%,pa_ref,qc_serien_nummer,qc_barcode_leak_test_values,p_product,p_product_full_name,p_product_group,p_product_type,qc_pa_beschichtungsjahr
count,118687,118348,118687,118686,118641,118682,118685,118687,118687,118687,43045.00,118687,118670,90978,118670,118670,118670,118670,118670
unique,1236,947,7,22,9,22,4,9,7,3,12.00,1263,117862,90322,22,22,2,8,2
top,d6dca86ea5f0,121,5200bdfc01a1,fb83fd553ff7,6d2830b1e76d,fb83fd553ff7,use,6d2830b1e76d,6d2830b1e76d,9bec1f36ec0d,0.06,f75dcbde8fe4,cc050207a397,2405709474110X0245XD32110,fb83fd553ff7,22227b31350f,4d6474389c69,6d2830b1e76d,W
freq,426,548,41352,24384,51354,24381,118537,51354,46474,61534,19059.00,426,2,2,24380,24380,112980,51353,62936


### float

In [217]:
df5.select_dtypes(float).head()

,pa_ps_beschichtete_rollenlange_m,pa_beschichtete_rollenlange_m,pa_bahn-geschwin-digkeit_m/min,pa_raum_temperatur_start_c,pa_temperatur_m-pda-bad_c,pa_cm-pda_0m,pa_cm-pda_500m,pa_cm-pda_1000m,pa_cm-pda_1500m,pa_cm-pda_2000m,...,sum_of_scrap,roll_position,pda_median,chlor_con_median,f_danfugt_bs_median,f_danfugt_as_median,f_codfely_median,temp_median,ps_f_auftragswerk_median,ps_dicke_median
0,250.0,4480.0,9.0,24.0,19.0,2.34,2.36,2.44,2.4,2.48,...,0.0,58.181818,1.23,826.0,58.0,58.0,80.1,22.0,78.0,134.5
1,250.0,4480.0,9.0,24.0,19.0,2.34,2.36,2.44,2.4,2.48,...,0.0,116.363636,1.23,826.0,58.0,58.0,80.1,22.0,78.0,134.5
2,250.0,4480.0,9.0,24.0,19.0,2.34,2.36,2.44,2.4,2.48,...,0.0,174.545455,1.23,826.0,58.0,58.0,80.1,22.0,78.0,134.5
3,250.0,4480.0,9.0,24.0,19.0,2.34,2.36,2.44,2.4,2.48,...,0.0,232.727273,1.23,826.0,58.0,58.0,80.1,22.0,78.0,134.5
4,250.0,4480.0,9.0,24.0,19.0,2.34,2.36,2.44,2.4,2.48,...,0.0,290.909091,1.23,826.0,58.0,58.0,80.1,22.0,78.0,134.5


In [246]:
# Check all columns that have missing values
naCols = df5.isna().any()
df5_nans = df5.loc[:,naCols]

# Check the percentage of missing values in each column


dfUNAN = pd.DataFrame({'nunique':df5.select_dtypes(float).nunique(dropna = True),
             'nan_%':df5.select_dtypes(float).isna().sum()/df5.shape[0]})

dfUNAN[dfUNAN['nunique'] < 11].sort_values(by = ['nunique'])


,nunique,nan_%
pa_ctmc_richtwert_%,1,0.998972
pa_ps_beschichtete_rollenlange_m,2,0.000000
pa_staub-sauger_ref_high_0_aus_>0_an,2,0.000000
ps_gap_micro_m,2,0.000000
pa_decan-trockner_geblase_ref_%,2,0.000000
qc_nachkommadurchfluss,2,0.000000
qc_breite,2,0.000000
qc_mstaerke,2,0.000000
sc_l_ergebnis_p_nio,2,0.000000
sc_l_parameter_pressure_absolut,2,0.000000


Issue:
- pa_ctmc_richtwert_% is also constant, since it has only 1 unique value. 99,8972% of its values are missing. Is that relevant for our data analysis or can we simply drop this column?

In [269]:
tmcCols = [s for s in df5.columns if 'tmc' in s]

print(tmcCols)

noNas   = ~df5['pa_ctmc_richtwert_%'].isna()

df5.loc[~noNas,tmcCols].describe(include = 'all')

# df5[['pa_tmc_gehalt_%']].describe()

['pa_ctmc_richtwert_%', 'pa_ctmc_%', 'pa_ndecan/tmc', 'pa_tmc_gehalt_%']


,pa_ctmc_richtwert_%,pa_ctmc_%,pa_ndecan/tmc,pa_tmc_gehalt_%
count,0.0,118565.000000,118565.000000,42923.00
unique,NaN,NaN,NaN,12.00
top,NaN,NaN,NaN,0.06
freq,NaN,NaN,NaN,19026.00
mean,NaN,0.254127,586.908683,NaN
std,NaN,3.277741,333.308706,NaN
min,NaN,0.000000,0.000000,NaN
25%,NaN,0.120000,293.000000,NaN
50%,NaN,0.120000,579.000000,NaN
75%,NaN,0.120000,873.000000,NaN


Check for columns with less than 10 unique values in order to find categoricals and booleans

In [27]:
#df5.apply(pd.unique, axis = 0)
#len(df5['pa_badwechsel_m-pda'].unique())
col = list(df5.columns)[2]
# for col in df5:


for colNr in range(347):
    unList = df5.iloc[:,colNr].unique() 
    nrUn = len(unList)
    if nrUn < 10:
        print('Column {} has {} unique values: {}'.format(colNr, nrUn, unList))


Column 1 has 2 unique values: [250. 280.]
Column 3 has 3 unique values: [ 9. 12.  6.]
Column 4 has 2 unique values: [0. 1.]
Column 5 has 2 unique values: [0. 1.]
Column 6 has 2 unique values: [1. 0.]
Column 7 has 2 unique values: [0. 1.]
Column 8 has 2 unique values: [0. 1.]
Column 20 has 7 unique values: [ 0.   16.6   2.4   4.    0.67  0.16  0.12]
Column 22 has 2 unique values: [ nan 0.12]
Column 25 has 8 unique values: [20.  21.  22.  19.  18.   0.  17.  22.4]
Column 52 has 9 unique values: [30. 35. 65. 25. 33.  0. 36. 27. 32.]
Column 73 has 9 unique values: [  0.  80.  85. 100.  65.   5.  88.  30.  92.]
Column 77 has 2 unique values: [  0. 100.]
Column 78 has 2 unique values: [  0. 100.]
Column 79 has 2 unique values: [  0. 100.]
Column 80 has 2 unique values: [  0. 100.]
Column 81 has 2 unique values: [  0. 100.]
Column 82 has 2 unique values: [9. 6.]
Column 84 has 6 unique values: [2.3 2.9 4.4 3.3 0.  5.6]
Column 85 has 7 unique values: [2.5  3.1  4.8  3.5  0.   6.   2.52]
Column 

In [28]:
int(1) == float(1) 

True

In [29]:
float(2.4)

2.4

In [30]:
df5.derived_date.unique()

array(['2018-05-01T00:00:00.000000000', '2018-07-01T00:00:00.000000000',
       '2018-10-01T00:00:00.000000000', '2018-11-01T00:00:00.000000000',
       '2019-03-01T00:00:00.000000000', '2019-05-01T00:00:00.000000000',
       '2019-06-01T00:00:00.000000000', '2019-07-01T00:00:00.000000000',
       '2018-01-01T00:00:00.000000000', '2018-02-01T00:00:00.000000000',
       '2018-04-01T00:00:00.000000000', '2018-06-01T00:00:00.000000000',
       '2018-09-01T00:00:00.000000000', '2019-02-01T00:00:00.000000000',
       '2019-04-01T00:00:00.000000000', '2019-09-01T00:00:00.000000000',
       '2018-12-01T00:00:00.000000000', '2019-10-01T00:00:00.000000000',
       '2018-03-01T00:00:00.000000000', '2019-01-01T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000', '2018-08-01T00:00:00.000000000',
                                 'NaT', '2019-08-01T00:00:00.000000000',
       '2019-12-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [231]:
# Check all the prefixes in our data

L = []
for s in list(df5.columns):
    prefix = (s.split('_',1))[0]
    L.append(prefix)
    
list(np.unique(np.array(L)))
    

['assembled',
 'assembling',
 'chlor',
 'ct',
 'ct1',
 'derived',
 'element',
 'f',
 'leak',
 'nr',
 'p',
 'pa',
 'pda',
 'pp',
 'ps',
 'qc',
 'qt',
 'reaction',
 'roll',
 'sc',
 'sum',
 'temp',
 'total',
 'winding']

Issues:

- [x] Column with name "windung" instead of "winding"

In [52]:
# Check all columns that have missing values
naCols = df5.isna().any()
df5_nans = df5.loc[:,naCols]

# Check the percentage of missing values in each column
(df5_nans.isna().sum()/df5.shape[0]).sort_values(ascending = False)


sc_l_datum_hand                 0.999360
pa_ctmc_richtwert_%             0.998972
pa_tmc_gehalt_in_percentage     0.637323
sc_l_datum_auto                 0.246573
qc_datum_leak_test_values       0.233463
sc_datum_generate               0.233463
qc_barcode_leak_test_values     0.233463
qc_einlager_datum               0.031562
qc_verpackungs_datum            0.030770
qt_datum                        0.020086
windung_begin_date              0.007271
ps_lsg                          0.002856
nr                              0.000876
winding_end_date                0.000556
pp_actual_product_short_name    0.000388
ps_date_coating                 0.000219
pp_plan_end_date_winding        0.000177
derived_date                    0.000143
qc_serien_nummer                0.000143
qc_pa_beschichtungsjahr         0.000143
sc_d_datum                      0.000143
qc_datum_product_properties     0.000143
p_product                       0.000143
p_product_full_name             0.000143
p_product_group 

In [ ]:
sc_l_datum_hand                 0.999360
pa_ctmc_richtwert_%             0.998972
pa_tmc_gehalt_in_percentage     0.637323
sc_l_datum_auto                 0.246573
qc_datum_leak_test_values       0.233463
sc_datum_generate               0.233463
qc_barcode_leak_test_values     0.233463

In [45]:
df5_nans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118687 entries, 0 to 118686
Data columns (total 31 columns):
pa_ctmc_richtwert_%             122 non-null float64
nr                              118583 non-null float64
ps_date_coating                 118661 non-null datetime64[ns]
ps_lsg                          118348 non-null object
qt_datum                        116303 non-null datetime64[ns]
pp_plan_product                 118686 non-null object
pp_plan_ausbeute_elemente       118678 non-null float64
pp_plan_end_date_winding        118666 non-null datetime64[ns]
pp_actual_product_short_name    118641 non-null object
pp_actual_product               118682 non-null object
pp_actual_usage                 118685 non-null object
windung_begin_date              117824 non-null datetime64[ns]
winding_end_date                118621 non-null datetime64[ns]
pa_tmc_gehalt_in_percentage     43045 non-null object
qc_serien_nummer                118670 non-null object
qc_erfassungs_datum      